# Sentiment Analysis of "The Lion King (2019)" movie reviews

## Web Scraping

In [8]:
import requests
import time
import pandas as pd
import json

###### Connect to rottentomatoes.com and collect user reviews for movie Lion king(2019).

#### Each session retrives 10 reviews. So run 300 iterations and have time gap of 5 seconds between each iteration. 
##### Save the result of each session as json file in 'reviews' folder. After the run completes we have 300 json files.

In [9]:
headers = {
'Referer': 'https://www.rottentomatoes.com/m/the_lion_king_2019/reviews?type=user',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36',
'X-Requested-With': 'XMLHttpRequest',
}
url = 'https://www.rottentomatoes.com/napi/movie/9057c2cf-7cab-317f-876f-e50b245ca76e/reviews/user'
payload = {
'direction': 'next',
'endCursor': '',
'startCursor': '',
}
s = requests.Session()

In [ ]:
headers = {
'Referer': 'https://www.rottentomatoes.com/m/the_lion_king_2019/reviews?type=user',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36',
'X-Requested-With': 'XMLHttpRequest',
}
url = 'https://www.rottentomatoes.com/napi/movie/9057c2cf-7cab-317f-876f-e50b245ca76e/reviews/user'
payload = {
'direction': 'next',
'endCursor': '',
'startCursor': '',
}
s = requests.Session()
i=1
while i<=300:
    r = s.get(url, headers=headers, params=payload)
    data = r.json()
    with open('reviews//review'+str(i)+'.json', 'w') as f:
        json.dump(data['reviews'], f)
    payload['endCursor'] = data['pageInfo']['endCursor']
    payload['startCursor'] = data['pageInfo']['startCursor']
    i+=1
    time.sleep(5)

# Combine json files to csv

In [1]:
import json
import os
from glob import glob
import pandas as pd

In [10]:
pattern = os.path.join('reviews', '*.json')

In [11]:
# This code combines reads all json files and writes the required features to a dataframe
data = pd.DataFrame()
for file_name in glob(pattern):
    with open(file_name) as f:
        json_data = json.load(f)
        df = pd.DataFrame.from_dict(json_data, orient='columns')
        user_data = pd.DataFrame()
        for review in json_data:
            user = pd.DataFrame([review['user']])
            user_data = user_data.append(user, ignore_index=True)
        df = pd.concat([df,user_data], axis=1)
        data = data.append(df, ignore_index=True)

In [12]:
data.head()

,createDate,displayImageUrl,displayName,hasProfanity,hasSpoilers,isSuperReviewer,isVerified,rating,review,score,timeFromCreation,updateDate,user,accountLink,displayName,realm,userId
0,2019-09-01T15:09:28.356Z,None,LD,False,False,False,True,STAR_4,Pretty scenery and movie really tried to captu...,4.0,56m ago,2019-09-01T15:09:28.356Z,{'userId': '4126eeea-0101-4115-b1b1-4eb0913a07...,None,LD,Fandango,4126eeea-0101-4115-b1b1-4eb0913a071b
1,2019-09-01T15:03:09.678Z,None,Trisha,False,False,False,True,STAR_3,it did not have the snap crackle pop that the ...,3.0,1h ago,2019-09-01T15:03:09.678Z,{'userId': '1F4B543E-C7B2-434A-83D0-64B3250043...,None,Trisha,Fandango,1F4B543E-C7B2-434A-83D0-64B3250043E2
2,2019-09-01T14:42:12.088Z,None,Oscar G,False,False,False,False,STAR_3,The live-action movie was a nice attempt at re...,3.0,1h ago,2019-09-01T14:42:12.666Z,"{'userId': '978220753', 'realm': 'RT', 'displa...",/user/id/978220753,Oscar G,RT,978220753
3,2019-09-01T12:58:19.986Z,None,None,False,False,False,False,STAR_4,go fmj. v v .knt mg mi n olmmrny mb s m mymhmr...,4.0,3h ago,2019-09-01T12:58:48.065Z,"{'userId': '260317136', 'realm': 'RT', 'displa...",/user/id/260317136,None,RT,260317136
4,2019-09-01T11:17:27.650Z,None,Volker D,False,False,False,False,STAR_1_5,"It´s like a cover of your favorite song, by th...",1.5,5h ago,2019-09-01T11:28:39.020Z,"{'userId': '978220514', 'realm': 'RT', 'displa...",/user/id/978220514,Volker D,RT,978220514


In [13]:
data.drop('user', axis=1, inplace=True)

In [14]:
# write the collected review data to a csv file, that can be loaded later for training
pd.DataFrame(data).to_csv("train_data.csv")